In [1]:
import os
os.getcwd()

'd:\\Coding\\War of the ring\\War-of-the-Ring\\notebook'

In [2]:
root = "../src/assets/map/"

In [3]:
import pygame
from PIL import Image
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, Tuple, List

pygame 2.6.1 (SDL 2.28.4, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# pygame window dimensions
WINDOW_WIDTH = 1920
WINDOW_HEIGHT = 1080

# map dimension
map_image = Image.open(root + "map_cut.png")
map_mask = Image.open(root + "map_mask.png")
MAP_WIDTH = map_image.width
MAP_HEIGHT = map_image.height

In [5]:
map_mask.getpixel((1005, 1005))[1:]

(14, 116, 255)

In [6]:
print(f"map width = {MAP_WIDTH}")
print(f"map height = {MAP_HEIGHT}")

map width = 6854
map height = 4708


In [7]:
mapheight_fit_factor = WINDOW_HEIGHT/MAP_HEIGHT*0.75

In [8]:
game_area = {
    "x0" : 100, "y0" : 0,
    "width" : mapheight_fit_factor*MAP_WIDTH, "height" : mapheight_fit_factor*MAP_HEIGHT 
}

In [9]:
class TextClass:
    def __init__(self, text_content : str , font_loc : str,
                 font_size : int, font_color : Tuple[int, int, int],
                 rect_center : Tuple[int, int]):
        self.text_content = text_content
        self.font_loc = font_loc
        self.font_size = font_size
        self.font_color = font_color
        self.font = pygame.font.Font(self.font_loc, self.font_size)
        self.text = self.font.render(self.text_content, True, self.font_color)
        self.text_rect = self.text.get_rect()
        self.text_rect.center = rect_center

    def draw_text(self, screen : pygame.Surface):
        screen.blit(self.text, self.text_rect)

class Quit_Box:
    def __init__(self, text_values : dict, 
                 x : int = 0, y : int = 0,
                 width : int = 100, height : int = 50,
                 ):
        # box
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        # text
        text_values['rect_center'] = (self.x + self.width/2, self.y + self.height/2)
        self.text = TextClass(**text_values)

    def draw(self, screen : pygame.Surface) -> None:
        # draw quit game box at top left
        pygame.draw.rect(screen, (255, 255, 255), pygame.Rect(self.x, self.y, self.width, self.height))
        self.text.draw_text(screen)

    def is_inside(self, mouse_x : int, mouse_y : int) -> bool:
        if mouse_x >= self.x and mouse_x <= self.x + self.width\
        and mouse_y >= self.y and mouse_y <= self.y + self.height:
            return True
        return False

In [10]:
def get_map_coordinate(mouse_x : int, mouse_y : int) -> Tuple[int, int]:
    map_x = (mouse_x - game_area['x0']) // mapheight_fit_factor 
    map_y = (mouse_y - game_area['y0']) // mapheight_fit_factor
    return int(map_x), int(map_y)

In [11]:
def rgb2hex(r : int, g : int, b : int) -> str:
    return "{:02x}{:02x}{:02x}".format(r, g, b)

In [12]:
def get_region_name(region_color : str, df : pd.DataFrame) -> str:
    return df[df['Color-Hex'] == region_color]['Name'].iloc[0]

In [13]:
df = pd.read_csv(root + "map_mask_data.csv")
df.head()

,Name,Type,Color-Decimal,Color-Hex
0,Anfalas,region,100,000064
1,Angmar,region,200,0000C8
2,Arnor,region,400,000190
3,Ash Mountains,region,500,0001F4
4,Barad Dur,region,600,000258


In [15]:
# Initialize Pygame
pygame.init()

# ui elements
text_values = {
    "text_content" : "Quit",
    "font_loc" : root + "font.ttf",
    "font_size" : 30, "font_color" : (0, 0, 0),
    "rect_center" : (0, 0)
}
quit_box = Quit_Box(text_values=text_values)

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT), pygame.FULLSCREEN, 32)
pygame.display.set_caption("War of the Ring")
map_image = pygame.image.load(root + "map_cut.png").convert()
map_image = pygame.transform.smoothscale_by(map_image, mapheight_fit_factor)
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            mouse_x, mouse_y = pygame.mouse.get_pos()
            if quit_box.is_inside(mouse_x, mouse_y):
                running = False
            elif mouse_x >= game_area['x0'] and mouse_x <= game_area['x0'] + game_area['width']\
            and mouse_y >= game_area['y0'] and mouse_y <= game_area['y0'] + game_area['height']:
                print("Inside game area")
                map_x, map_y = get_map_coordinate(mouse_x, mouse_y)
                region_color = map_mask.getpixel((map_x, map_y))[:3]
                region_color_hex = rgb2hex(*region_color)
                print(f"Region Color : {region_color_hex}")
                region_name = get_region_name(region_color_hex.upper(), df)
                print(f"Region name : {region_name}")
            else:
                print("Outside game area")

    screen.fill((0, 0, 0))

    # render the frame
    # map
    screen.blit(map_image, (100, 0))
    # ui
    quit_box.draw(screen)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

Inside game area
Region Color : 00238c
Region name : The Shire
Inside game area
Region Color : 0f517c
Region name : City The Shire FP
Inside game area
Region Color : 0f5690
Region name : Settlemet Marker The Shire FP
Inside game area
Region Color : 0f4240
Region name : Strategy Deck FP
Inside game area
Region Color : 0f42a4
Region name : Character Deck FP
Inside game area
Region Color : 0f6ec8
Region name : Character Deck SH
Inside game area
Region Color : 0f6e64
Region name : Strategy Deck SH
Inside game area
Region Color : 001450
Region name : Minas Morgul
Inside game area
Region Color : 0f4e5c
Region name : Strong Hold Map Minas Morgul SH
Inside game area
Region Color : 0f5a78
Region name : Settlemet Marker Minas Morgul SH
Inside game area
Region Color : 000000
Region name : Background
